<a href="https://colab.research.google.com/github/khalidpark/deeplearning_whitepaper/blob/main/deep_learning_whitepaper_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CV Cross Validation

In [18]:
# 데이터를 불러옵니다. 
from tensorflow.keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()


In [20]:
print(x_train[:2])
print(y_train[:2])

[[1.23247e+00 0.00000e+00 8.14000e+00 0.00000e+00 5.38000e-01 6.14200e+00
  9.17000e+01 3.97690e+00 4.00000e+00 3.07000e+02 2.10000e+01 3.96900e+02
  1.87200e+01]
 [2.17700e-02 8.25000e+01 2.03000e+00 0.00000e+00 4.15000e-01 7.61000e+00
  1.57000e+01 6.27000e+00 2.00000e+00 3.48000e+02 1.47000e+01 3.95380e+02
  3.11000e+00]]
[15.2 42.3]


In [21]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import KFold, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

kf = KFold(n_splits = 5)             
skf = StratifiedKFold(n_splits = 5, random_state = 100, shuffle = True) 

x_train.shape


(404, 13)

In [22]:
x_train.shape[0]


404

In [32]:
x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)
for train_index, val_index in kf.split(np.zeros(x_train.shape[0])):
  training_data = x_train.iloc[train_index, :]
  training_data_label = y_train.iloc[train_index]
  validation_data = x_train.iloc[val_index, :]
  validation_data_label = y_train.iloc[val_index]

  # CV
  model.compile(loss='mean_squared_error', optimizer='adam')
  model.fit(x_train, y_train,
			    epochs=10,
          batch_size=30,
          validation_data = (validation_data, validation_data_label),
          )
  results = model.evaluate(x_test, y_test, batch_size=128)
  print("test loss, test mse:", results)

Epoch 1/10
14/14 [==============================] - 1s 17ms/step - loss: 182.6682 - val_loss: 68.7254
Epoch 2/10
14/14 [==============================] - 0s 4ms/step - loss: 69.7003 - val_loss: 57.2732
Epoch 3/10
14/14 [==============================] - 0s 4ms/step - loss: 61.2859 - val_loss: 46.2394
Epoch 4/10
14/14 [==============================] - 0s 4ms/step - loss: 54.5046 - val_loss: 44.9849
Epoch 5/10
14/14 [==============================] - 0s 4ms/step - loss: 55.7428 - val_loss: 40.4928
Epoch 6/10
14/14 [==============================] - 0s 5ms/step - loss: 47.3430 - val_loss: 43.6442
Epoch 7/10
14/14 [==============================] - 0s 4ms/step - loss: 53.2663 - val_loss: 39.9225
Epoch 8/10
14/14 [==============================] - 0s 4ms/step - loss: 51.6695 - val_loss: 40.3026
Epoch 9/10
14/14 [==============================] - 0s 4ms/step - loss: 59.5627 - val_loss: 35.4440
Epoch 10/10
1/1 [==============================] - 0s 69ms/step - loss: 47.3767
test loss, test ms

# 입력데이터 정규화

In [33]:
# 정규화를 위한 함수 호출
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
print(x_train[:10])


[[-0.27224633 -0.48361547 -0.43576161 -0.25683275 -0.1652266  -0.1764426
   0.81306188  0.1166983  -0.62624905 -0.59517003  1.14850044  0.44807713
   0.8252202 ]
 [-0.40342651  2.99178419 -1.33391162 -0.25683275 -1.21518188  1.89434613
  -1.91036058  1.24758524 -0.85646254 -0.34843254 -1.71818909  0.43190599
  -1.32920239]
 [ 0.1249402  -0.48361547  1.0283258  -0.25683275  0.62864202 -1.82968811
   1.11048828 -1.18743907  1.67588577  1.5652875   0.78447637  0.22061726
  -1.30850006]
 [-0.40149354 -0.48361547 -0.86940196 -0.25683275 -0.3615597  -0.3245576
  -1.23667187  1.10717989 -0.51114231 -1.094663    0.78447637  0.44807713
  -0.65292624]
 [-0.0056343  -0.48361547  1.0283258  -0.25683275  1.32861221  0.15364225
   0.69480801 -0.57857203  1.67588577  1.5652875   0.78447637  0.3898823
   0.26349695]
 [-0.37502238 -0.48361547 -0.54747912 -0.25683275 -0.54935658 -0.78865126
   0.18954148  0.48371503 -0.51114231 -0.71552978  0.51145832  0.38669063
  -0.13812828]
 [ 0.58963463 -0.48361547

# 모델 자동 검증 기능

In [34]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 중요한 하이퍼 파라미터들
inputs = x_train.shape[1]
epochs = 75                 # 전체 반복횟수
batch_size = 10             # 한번에 학습하는 사이즈


# 모델을 생성합니다
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# Sequential인 경우, 아래의 방법으로도 모델을 만들 수 있습니다.
# model = Sequential(
# [
#     Dense(64, activation='relu', input_shape=(inputs,)),
#     Dense(64, activation='relu'),
#     Dense(1)
# ]
# )

# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

# Fit Model
model.fit(x_train, y_train, 
          validation_data=(x_test,y_test),  # validation set
          epochs=epochs,                    # 전체 반복횟수
          batch_size=batch_size             # 한번에 학습하는 사이즈
         )


Epoch 1/75
41/41 [==============================] - 1s 6ms/step - loss: 521.7471 - mse: 521.7471 - mae: 21.1969 - val_loss: 408.7388 - val_mse: 408.7388 - val_mae: 18.2125
Epoch 2/75
41/41 [==============================] - 0s 2ms/step - loss: 321.4866 - mse: 321.4866 - mae: 15.6391 - val_loss: 107.5910 - val_mse: 107.5910 - val_mae: 8.6594
Epoch 3/75
41/41 [==============================] - 0s 2ms/step - loss: 61.9244 - mse: 61.9244 - mae: 6.0047 - val_loss: 43.5690 - val_mse: 43.5690 - val_mae: 5.0027
Epoch 4/75
41/41 [==============================] - 0s 3ms/step - loss: 29.3929 - mse: 29.3929 - mae: 3.9653 - val_loss: 32.6122 - val_mse: 32.6122 - val_mae: 4.4358
Epoch 5/75
41/41 [==============================] - 0s 2ms/step - loss: 20.4711 - mse: 20.4711 - mae: 3.2108 - val_loss: 28.4624 - val_mse: 28.4624 - val_mae: 4.2045
Epoch 6/75
41/41 [==============================] - 0s 2ms/step - loss: 26.7205 - mse: 26.7205 - mae: 3.5058 - val_loss: 26.7852 - val_mse: 26.7852 - val_mae: 